### Step 1: Import Libraries

In [1]:
# Predictive models.
from prophet import Prophet
from mlforecast import MLForecast
from xgboost import XGBRegressor
from window_ops.rolling import rolling_mean, rolling_max, rolling_min

# Feature engineering.
from mlforecast.feature_engineering import transform_exog

# Data processing.
import numpy as np
import pandas as pd
import datetime as dt
import itertools

# Ignore warning messages.
import warnings

# Save model.
import pickle

# Model performance evaluation.
from sklearn.metrics import mean_absolute_error

C:\Users\peter\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


### Step 2: Model Selection

In the csv file, we have multiple predictor variables. Therefore, it's a multivariate time series.

Hence, this problem can be categorized as **multivariate time series with seasonalities**.

Therefore, we are going to use library **Prophet** and **mlforecast** to create our models.

### Step 3: Prophet Model Creation

#### Step 3.1: Data Preprocessing for Prophet models

In [2]:
data = pd.read_csv('Cleaned Dataset.csv')
# Rename the columns for Prophet training purposes.
data.rename(columns={'Date_Time':'ds', 'Ontario_Demand': 'y'}, inplace=True)
data.head()

,Weekday,HOEP,y,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station,ds
0,0,0.49000,14023.0,-0.30,-5.620,20.0,-3.180,70.0,-5.10,99.67,2016-01-01 00:00:00
1,0,0.55375,13417.0,-0.30,-6.300,25.0,-3.480,68.0,-5.50,99.63,2016-01-01 01:00:00
2,0,0.61750,12968.0,-0.40,-6.550,26.0,-3.430,73.0,-4.70,99.59,2016-01-01 02:00:00
3,0,0.68125,13092.7,-0.57,-6.767,25.7,-3.573,73.3,-4.81,99.57,2016-01-01 03:00:00
4,0,0.74500,13217.4,-0.74,-6.984,25.4,-3.716,73.6,-4.92,99.55,2016-01-01 04:00:00


#### Step 3.2 Create and Evaluate Prophet Baseline Model

In [3]:
def warm_start_params(m):
    """
    Retrieve parameters from a trained model in the format used to initialize a new Stan model.
    Note that the new Stan model must have these same settings:
        n_changepoints, seasonality features, mcmc sampling
    for the retrieved parameters to be valid for the new model.

    Parameters
    ----------
    m: A trained model of the Prophet class.

    Returns
    -------
    A Dictionary containing retrieved parameters of m.
    """
    res = {}
    for pname in ['k', 'm', 'sigma_obs']:
        if m.mcmc_samples == 0:
            res[pname] = m.params[pname][0][0]
        else:
            res[pname] = np.mean(m.params[pname])
    for pname in ['delta', 'beta']:
        if m.mcmc_samples == 0:
            res[pname] = m.params[pname][0]
        else:
            res[pname] = np.mean(m.params[pname], axis=0)
    return res

first_train_end_date = '2020-06-30 23:00:00'
first_test_end_date = '2020-07-01 23:00:00'

train = data[data['ds'] <= str(first_train_end_date)]
test = data[(data['ds'] > str(first_train_end_date)) & (data['ds'] <= str(first_test_end_date))]

# Initiate the Prophet model.
model_baseline = Prophet()

# Save the baseline model for warm start.
m1 = model_baseline.fit(train)

15:20:18 - cmdstanpy - INFO - Chain [1] start processing
15:20:32 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
# Split the date by every day.
train_end_date = pd.date_range("2020-06-30 23:00:00", periods=62, freq="d")
test_end_date = pd.date_range("2020-07-01 23:00:00", periods=62, freq="d")

total_baseline_MAE = 0
count = 0

# Looping through every day of July and August.
for i, _ in enumerate(train_end_date):
    # New training and testing data.
    train = data[data['ds'] <= str(train_end_date[i])]
    test = data[(data['ds'] > str(train_end_date[i])) & (data['ds'] <= str(test_end_date[i]))]

    m = Prophet()
    m.fit(train, init = warm_start_params(m1))

    # Make prediction.
    f = m.predict(pd.DataFrame(test))

    # Check MAE value.
    performance_baseline_MAE = mean_absolute_error(test['y'], f['yhat'])
    count += 1
    total_baseline_MAE += performance_baseline_MAE
    print(f'MAE at {test_end_date[i]} for baseline model is {performance_baseline_MAE}')
    
print(f'MAE for the baseline model is {total_baseline_MAE / count}')
warnings.simplefilter(action='ignore', category=FutureWarning)

Takes too long to run the code block, therefore results will be omitted. MAE around 1700.

#### Step 3.3 Create and Evaluate Multivariate Model (with weekend seasonality and lag)

In [4]:
# Check correlations of between predictor variables and response variable.
data.drop(columns = ['ds']).corrwith(data['y'])

Weekday             -0.230610
HOEP                 0.031180
y                    1.000000
Temperature          0.058032
Windchill_Index      0.046275
Wind_Speed           0.124627
Humidex              0.082014
Relative_Humidity   -0.157249
Dew_Point            0.001706
Pressure_Station    -0.018048
dtype: float64

Choose Relative_Humidity, Wind_Speed, Humidex and Weekday_Binary as predictor variables.

In [ ]:
# Split the date by every hour.
train_end_date = pd.date_range("2020-06-30 23:00:00", periods=62, freq="d")
test_end_date = pd.date_range('2020-07-01 23:00:00', periods=62, freq="d")

total_multivariate_MAE = 0
count = 0

# Looping through every day of July and August.
for i, _ in enumerate(train_end_date):
    # New training and testing data.
    train = data[data['ds'] <= str(train_end_date[i])]
    test = data[(data['ds'] > str(train_end_date[i])) & (data['ds'] <= str(test_end_date[i]))]

    for j in range(24):
        # Initiate the Prophet model, disable all seasonalities except for daily.
        mm = Prophet(weekly_seasonality = False, yearly_seasonality = False)
        
        # Add predictor variables.
        mm.add_regressor('Relative_Humidity', standardize=False)
        mm.add_regressor('Wind_Speed', standardize=False)
        mm.add_regressor('Humidex', standardize=False)
        mm.add_regressor('Weekday_Binary', standardize=False)
        
        # Fit the model with lagged data.
        mm.fit(train.shift(24-j)[24-j:])
        
        # Make prediction.
        fm = mm.predict(pd.DataFrame(test.iloc[j]).transpose())
        
        # Check MAE value.
        performance_multivariate_MAE = mean_absolute_error([test['y'].iloc[j]], fm['yhat'])
        total_multivariate_MAE += performance_multivariate_MAE
        count += 1 
        print(f'The MAE at {test["ds"].iloc[j]} for multivariate model is {performance_multivariate_MAE}')
        
print(f'The total MAE for the multiariate model is {total_multivariate_MAE / count}')

warnings.simplefilter(action='ignore', category=FutureWarning)

Takes too long to run the code block, therefore results will be omitted. MAE around 1600.

### Step 4: Create XGBoost Model

Since the Prophet models can only capture additive and multiplicative relationships between x and y.

Therefore we are going to try XGBoost now since the results were not promising and XGBoost can capture non-linear relationships effectively.

#### Step 4.1 Data Preprocessing

In [5]:
data = pd.read_csv('Cleaned Dataset.csv')
# Rename the columns for mlforecast training purposes.
data.rename(columns={'Date_Time':'ds', 'Ontario_Demand': 'y'}, inplace=True)
data['ds'] = pd.to_datetime(data['ds'])
data['unique_id'] = 'id_00'
data.head()

,Weekday,HOEP,y,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station,ds,unique_id
0,0,0.49000,14023.0,-0.30,-5.620,20.0,-3.180,70.0,-5.10,99.67,2016-01-01 00:00:00,id_00
1,0,0.55375,13417.0,-0.30,-6.300,25.0,-3.480,68.0,-5.50,99.63,2016-01-01 01:00:00,id_00
2,0,0.61750,12968.0,-0.40,-6.550,26.0,-3.430,73.0,-4.70,99.59,2016-01-01 02:00:00,id_00
3,0,0.68125,13092.7,-0.57,-6.767,25.7,-3.573,73.3,-4.81,99.57,2016-01-01 03:00:00,id_00
4,0,0.74500,13217.4,-0.74,-6.984,25.4,-3.716,73.6,-4.92,99.55,2016-01-01 04:00:00,id_00


#### Step 4.2 Feature Engineering

In [6]:
# Add lag 1 and lag 24 for all the features.
transformed_features = transform_exog(data.drop(columns = ['Weekday']), lags=[1, 24])
transformed_data = data[['unique_id', 'ds']].merge(transformed_features, on=['unique_id', 'ds'])
transformed_data.head()

,unique_id,ds,HOEP,y,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,...,Wind_Speed_lag1,Wind_Speed_lag24,Humidex_lag1,Humidex_lag24,Relative_Humidity_lag1,Relative_Humidity_lag24,Dew_Point_lag1,Dew_Point_lag24,Pressure_Station_lag1,Pressure_Station_lag24
0,id_00,2016-01-01 00:00:00,0.49000,14023.0,-0.30,-5.620,20.0,-3.180,70.0,-5.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id_00,2016-01-01 01:00:00,0.55375,13417.0,-0.30,-6.300,25.0,-3.480,68.0,-5.50,...,20.0,NaN,-3.180,NaN,70.0,NaN,-5.10,NaN,99.67,NaN
2,id_00,2016-01-01 02:00:00,0.61750,12968.0,-0.40,-6.550,26.0,-3.430,73.0,-4.70,...,25.0,NaN,-3.480,NaN,68.0,NaN,-5.50,NaN,99.63,NaN
3,id_00,2016-01-01 03:00:00,0.68125,13092.7,-0.57,-6.767,25.7,-3.573,73.3,-4.81,...,26.0,NaN,-3.430,NaN,73.0,NaN,-4.70,NaN,99.59,NaN
4,id_00,2016-01-01 04:00:00,0.74500,13217.4,-0.74,-6.984,25.4,-3.716,73.6,-4.92,...,25.7,NaN,-3.573,NaN,73.3,NaN,-4.81,NaN,99.57,NaN


In [7]:
# Split the date by every hour.
train_end_date = pd.date_range("2020-06-30 23:00:00", periods=62, freq="d")
test_end_date = pd.date_range('2020-07-01 23:00:00', periods=62, freq="d")

total_mae = 0

# Looping through every day of July and August.
for i, _ in enumerate(train_end_date):
    # New training and testing data.
    train = transformed_data[transformed_data['ds'] <= str(train_end_date[i])]
    test = transformed_data[(transformed_data['ds'] > str(train_end_date[i])) & (transformed_data['ds'] <= str(test_end_date[i]))]

    train.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)

    xgb_model = [XGBRegressor(
        n_estimators = 100,
        random_state = 42
    )]

    model = MLForecast(models=xgb_model,
                       freq='h',
                       lags=[1, 24],
                       lag_transforms={
                           1: [(rolling_mean, 24), (rolling_max, 24), (rolling_min, 24)],
                       },
                       num_threads=6)

    model.preprocess(train, dropna=True)
    
    model.fit(train, id_col='unique_id', time_col='ds', target_col='y', static_features=[])
    
    predictions = model.predict(24, X_df = test)

    curr_mae = mean_absolute_error(test['y'], predictions['XGBRegressor'])
    total_mae += curr_mae

print(f'MAE for XGBoost is {total_mae / 62}.') 

MAE for XGBoost is 400.3597337729615.


#### Step 4.3 Hyperparameter Tuning

Use gridSearch to find the optimal parameters for XGBoost.

In [8]:
# Split the date by every hour.
train_end_date = pd.date_range("2020-06-30 23:00:00", periods=62, freq="d")
test_end_date = pd.date_range('2020-07-01 23:00:00', periods=62, freq="d")

param_grid = {'nthread':[4],
              'objective':['reg:squarederror'],
              'learning_rate': [.07, 0.1], 
              'max_depth': [5, 7],
              'min_child_weight': [2, 3, 4],
              'verbosity': [1],
              'random_state': [42]
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
maes = []  # Store the MAEs for each params here

# Loop through all combinations of parameters.
for params in all_params:
    
    model_mae = 0

    # Looping through every hour of July and August.
    for i, _ in enumerate(train_end_date):
        # New training and testing data.
        train = transformed_data[transformed_data['ds'] <= str(train_end_date[i])]
        test = transformed_data[(transformed_data['ds'] > str(train_end_date[i])) & (transformed_data['ds'] <= str(test_end_date[i]))]
    
        train.reset_index(drop = True, inplace = True)
        test.reset_index(drop = True, inplace = True)
    
        xgb_model = [XGBRegressor(
            **params
        )]
    
        model = MLForecast(models=xgb_model,
                           freq='h',
                           lags=[1, 24],
                           lag_transforms={
                               1: [(rolling_mean, 24), (rolling_max, 24), (rolling_min, 24)],
                           },
                           num_threads=6)
    
        model.preprocess(train, dropna=True)
        
        model.fit(train, id_col='unique_id', time_col='ds', target_col='y', static_features=[])
        
        predictions = model.predict(24, X_df = test)
    
        curr_mae = mean_absolute_error(test['y'], predictions['XGBRegressor'])
        model_mae += curr_mae

    maes.append(model_mae)

In [9]:
# Find the best parameters.
tuning_results = pd.DataFrame(all_params)
tuning_results['mae'] = [mae/62 for mae in maes]
tuning_results

,nthread,objective,learning_rate,max_depth,min_child_weight,verbosity,random_state,mae
0,4,reg:squarederror,0.07,5,2,1,42,410.974285
1,4,reg:squarederror,0.07,5,3,1,42,412.355180
2,4,reg:squarederror,0.07,5,4,1,42,411.158815
3,4,reg:squarederror,0.07,7,2,1,42,385.894091
4,4,reg:squarederror,0.07,7,3,1,42,383.163966
5,4,reg:squarederror,0.07,7,4,1,42,385.125813
6,4,reg:squarederror,0.10,5,2,1,42,402.184284
7,4,reg:squarederror,0.10,5,3,1,42,404.828985
8,4,reg:squarederror,0.10,5,4,1,42,399.573501
9,4,reg:squarederror,0.10,7,2,1,42,375.729226


Use learning_rate = 0.1, max_depth = 7, min_child_weight = 3 for final model.